# 如何获取对数概率

:::info 预备知识

本指南假定您已熟悉以下概念：

- [聊天模型](/docs/concepts/chat_models)

:::

某些聊天模型可以配置为返回令牌级别的对数概率，以表示给定令牌的可能性。本指南将介绍如何在LangChain中获取此信息。

## OpenAI

安装 `@langchain/openai` 包并设置您的 API 密钥：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/openai @langchain/core
</Npm2Yarn>
```

要让 OpenAI API 返回对数概率，我们需要将 `logprobs` 参数设置为 `true`。然后，对数概率会作为 `response_metadata` 的一部分包含在每个输出 [`AIMessage`](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html) 中：

In [1]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  model: "gpt-4o",
  logprobs: true,
});

const responseMessage = await model.invoke("how are you today?");

responseMessage.response_metadata.logprobs.content.slice(0, 5);

[
  {
    token: "Thank",
    logprob: -0.70174205,
    bytes: [ 84, 104, 97, 110, 107 ],
    top_logprobs: []
  },
  {
    token: " you",
    logprob: 0,
    bytes: [ 32, 121, 111, 117 ],
    top_logprobs: []
  },
  {
    token: " for",
    logprob: -0.000004723352,
    bytes: [ 32, 102, 111, 114 ],
    top_logprobs: []
  },
  {
    token: " asking",
    logprob: -0.0000013856493,
    bytes: [
       32,  97, 115,
      107, 105, 110,
      103
    ],
    top_logprobs: []
  },
  {
    token: "!",
    logprob: -0.00030102333,
    bytes: [ 33 ],
    top_logprobs: []
  }
]

并且也是流式消息块的一部分：

In [2]:
let count = 0;
const stream = await model.stream("How are you today?");
let aggregateResponse;

for await (const chunk of stream) {
  if (count > 5) {
    break;
  }
  if (aggregateResponse === undefined) {
    aggregateResponse = chunk;
  } else {
    aggregateResponse = aggregateResponse.concat(chunk);
  }
  console.log(aggregateResponse.response_metadata.logprobs?.content);
  count++;
}

[]
[
  {
    token: "Thank",
    logprob: -0.23375113,
    bytes: [ 84, 104, 97, 110, 107 ],
    top_logprobs: []
  }
]
[
  {
    token: "Thank",
    logprob: -0.23375113,
    bytes: [ 84, 104, 97, 110, 107 ],
    top_logprobs: []
  },
  {
    token: " you",
    logprob: 0,
    bytes: [ 32, 121, 111, 117 ],
    top_logprobs: []
  }
]
[
  {
    token: "Thank",
    logprob: -0.23375113,
    bytes: [ 84, 104, 97, 110, 107 ],
    top_logprobs: []
  },
  {
    token: " you",
    logprob: 0,
    bytes: [ 32, 121, 111, 117 ],
    top_logprobs: []
  },
  {
    token: " for",
    logprob: -0.000004723352,
    bytes: [ 32, 102, 111, 114 ],
    top_logprobs: []
  }
]
[
  {
    token: "Thank",
    logprob: -0.23375113,
    bytes: [ 84, 104, 97, 110, 107 ],
    top_logprobs: []
  },
  {
    token: " you",
    logprob: 0,
    bytes: [ 32, 121, 111, 117 ],
    top_logprobs: []
  },
  {
    token: " for",
    logprob: -0.000004723352,
    bytes: [ 32, 102, 111, 114 ],
    top_logprobs: []
  },
  {
   

## `topLogprobs`

如需查看每一步的其他潜在生成结果，可以使用 `topLogprobs` 参数：

In [3]:
const modelWithTopLogprobs = new ChatOpenAI({
  model: "gpt-4o",
  logprobs: true,
  topLogprobs: 3,
});

const res = await modelWithTopLogprobs.invoke("how are you today?");

res.response_metadata.logprobs.content.slice(0, 5);

[
  {
    token: "I'm",
    logprob: -2.2864406,
    bytes: [ 73, 39, 109 ],
    top_logprobs: [
      {
        token: "Thank",
        logprob: -0.28644064,
        bytes: [ 84, 104, 97, 110, 107 ]
      },
      {
        token: "Hello",
        logprob: -2.0364406,
        bytes: [ 72, 101, 108, 108, 111 ]
      },
      { token: "I'm", logprob: -2.2864406, bytes: [ 73, 39, 109 ] }
    ]
  },
  {
    token: " just",
    logprob: -0.14442946,
    bytes: [ 32, 106, 117, 115, 116 ],
    top_logprobs: [
      {
        token: " just",
        logprob: -0.14442946,
        bytes: [ 32, 106, 117, 115, 116 ]
      },
      { token: " an", logprob: -2.2694294, bytes: [ 32, 97, 110 ] },
      {
        token: " here",
        logprob: -4.0194297,
        bytes: [ 32, 104, 101, 114, 101 ]
      }
    ]
  },
  {
    token: " a",
    logprob: -0.00066632946,
    bytes: [ 32, 97 ],
    top_logprobs: [
      { token: " a", logprob: -0.00066632946, bytes: [ 32, 97 ] },
      {
        token: " li

## 下一步

你现在已了解如何在 LangChain 中从 OpenAI 模型获取 logprobs。

接下来，请查看本节中有关聊天模型的其他操作指南，例如 [如何让模型返回结构化输出](/docs/how_to/structured_output) 或 [如何跟踪令牌使用情况](/docs/how_to/chat_token_usage_tracking)。